In [1]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
!pip install -q datasets~=3.2.0 pydantic~=2.10.4
!pip install -q peft~=0.14.0 evaluate~=0.4.3 rouge_score~=0.1.2

# 訓練 PII 遮掩模型

在這個筆記本中，我們將展示如何使用 `transformers` 套件訓練 PII (個人識別資訊) 遮掩模型。我們將使用 `transformers` 套件中的 [`Seq2SeqTrainer`](https://huggingface.co/docs/evaluate/transformers_integrations#seq2seqtrainer) 類別來微調一個 Encoder-Decoder 架構的 [Flan T5](https://huggingface.co/docs/transformers/model_doc/t5) 模型。

> Flan-T5: Flan is a pretraining methods that is based on prompting.

In [ ]:
import pandas as pd
import numpy as np
import evaluate

from transformers import (
  AutoTokenizer,
  DataCollatorForSeq2Seq,
  T5ForConditionalGeneration,
  Seq2SeqTrainingArguments,
  Seq2SeqTrainer,
)
from datasets import load_dataset, DatasetDict
from transformers import (
  pipeline,
)

from typing import Any
from pydantic import BaseModel
from pprint import pprint

import torch

# 載入 PEFT 相關套件
from peft import LoraConfig, TaskType, PeftModel, get_peft_model

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

## 下載資料

In [3]:
# The full `train` split, only 25% of dataset
immutable_dataset = load_dataset("ai4privacy/pii-masking-65k", split="train[:10%]")

  ### 資料包含什麼？

In [ ]:
# 顯示原始資料中包含的 features 以及筆數
immutable_dataset

In [ ]:
# 檢視資料集中的第一筆資料
pd.set_option('display.max_colwidth', None)
pd.DataFrame(immutable_dataset[:1])

這個表格結構，包含四個欄位，分別是：

* `masked_text`: 這是一個包含 PII 遮掩的文本，我們將使用這個文本來訓練模型。

* `unmasked_text`: 這是一個包含 PII 資訊的原始文本，我們將使用這個文本來訓練模型。

其餘 `token_entity_labels` 及 `tokenised_unmasked_text` 是我們在訓練過程中不會使用到的欄位。


### 資料前處理

方便演示及訓練，我們將對資料進行以下前處理：

1. 保留 `masked_text` 及 `unmasked_text` 欄位。
2. 將資料及分為訓練集，驗證集及測試集。

In [ ]:
# 保留必要 features: 'masked_text', 'unmasked_text'
dataset = immutable_dataset.remove_columns(['token_entity_labels', 'tokenised_unmasked_text'])
# 顯示處理後的資料
dataset

In [7]:
# Reserve 0.1% of the training set for testing
test_dataset = dataset.train_test_split(
  test_size=0.001, # 0.1% of the data is used for testing
  shuffle=False, # Ensure that train and validation sets are the same across runs
  )
# Split into 80% training and 20% validation sets
train_dataset = test_dataset['train'].train_test_split(
  test_size=0.2, # 20% of the data is used for validation
  shuffle=False, # Ensure that train and test sets are the same across runs
  )
dataset = DatasetDict({
  'train': train_dataset['train'],
  'validation': train_dataset['test'],
  'test': test_dataset['test'],
  })

In [ ]:
# 顯示前 first_n_data 筆資料
first_n_data = 3
pd.set_option('display.max_colwidth', None)
pd.DataFrame(dataset['train'].select(range(first_n_data)))

## 訓練設定

### 批次大小 (Batch Size) 和 梯度累積步數 (Gradient Accumulation Steps)

批次大小（batch size）和梯度累積步數（gradient accumulation steps）之間的關係可以簡單地說明如下：

* 批次大小（batch size）：每次訓練迭代中使用的樣本數量。較大的批次大小通常需要更多的內存。
* 梯度累積步數（gradient accumulation steps）：在更新模型權重之前累積梯度的迭代次數。這允許使用較小的批次大小來模擬較大的批次大小。

當內存限制無法直接使用大批次大小時，可以通過梯度累積來實現。例如：

* 如果批次大小是 8，梯度累積步數是 4，這相當於使用批次大小為 32（8 * 4）進行訓練。

這樣可以在內存有限的情況下實現大批次大小的效果。

### 半精度浮點數

半精度訓練（Half-Precision Training）是一種使用 16 位浮點數（FP16）而不是 32 位浮點數（FP32）來訓練神經網絡的方法。這種方法的主要優點包括：

* 減少內存使用：FP16 數據類型佔用的內存比 FP32 少一半，允許在相同的硬件上訓練更大的模型或使用更大的批次大小。
* 加速計算：許多現代 GPU 對 FP16 計算進行了優化，可以更快地執行 FP16 運算，從而加速訓練過程。
* 節省帶寬：減少內存和帶寬的使用，有助於提高數據傳輸效率。

BFP16 (Brain Floating Point 16)
BFP16 是一種 16 位浮點數格式，主要由 Google 用於其 TPU（Tensor Processing Unit）。BFP16 的優點是它保留了與 FP32 相同的指數範圍，但尾數精度較低，這在某些情況下可以提供更好的數值穩定性。

FP16 (Half-Precision Floating Point)
FP16 是一種標準的 16 位浮點數格式，廣泛用於 GPU 加速的深度學習訓練。FP16 的優點是內存佔用少，計算速度快，但指數範圍和尾數精度都比 FP32 小。

![](https://miro.medium.com/v2/0*HapPSei5sok65wcv)

總體來說，半精度訓練可以在不顯著影響模型性能的情況下，提高訓練效率和資源利用率。

### 訓練設定

In [9]:
# 訓練相關設定
class Config(BaseModel):
  model_name: str = 'google/flan-t5-base' # Fine-tuned Language Network with T5
  torch_dtype: Any = torch.bfloat16 # 半精度浮點數
  adam_epsilon: float = 1e-4 # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  saved_model_path: str = 'sample_data/saved_encode_decode_model' # path to save the trained model
  saved_lora_path: str = 'sample_data/saved_encode_decode_lora_model' # path to save the trained LORA model
  batch_size: int = 2 # size of the input batch in training and evaluation
  gradient_accumulation_steps: int = 2 # number of updates steps to accumulate before performing a backward/update pass
  epochs: int = 3 # number of times to iterate over the entire training dataset
  lr: float = 2e-4 # learning rate, controls how fast or slow the model learns
  weight_decay: float = 0.01 # weight decay, helps the model stay simple and avoid overfitting by penalizing large weights.
  eval_metric: str = 'bleu' # evaluation metric

  # 文本生成相關設定
  temperature: float = 0.1 # temperature for sampling
  max_new_tokens: int = 125 # 限制最大生成字數
  repetition_penalty: float = 1.5 # 重複機率, 1~2 之間, 1.0 (no penalty), 2.0 (maximum penalty)

  # LORA 相關設定
  rank: int = 128 # rank of the Lora layers
  lora_alpha: int = rank * 2 # alpha for Lora scaling.
  lora_dropout: float = 0.05 # dropout probability for Lora layers

if device.type == 'mps': # 方便在 Apple Silicon 上快速測試
  config = Config(
    torch_dtype=torch.float32, # 在 Apple Silicon 需要使用全精度浮點數，否則會出現錯誤
  )
else:
  config = Config()

## Fine-tuning 前的表現

### 載入預訓練分詞器 (Tokenizer)

In [ ]:
# 透過預訓練模型取得 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
  config.model_name,
)
pprint(tokenizer)

In [ ]:
# 檢視 Tokenizer，是否存在 padding token 及 padding side 等資訊
pprint(tokenizer.pad_token)

In [ ]:
pprint(tokenizer.padding_side)

* 如果沒有定義 `pad_token`，請定義一個 `pad_token`，並將其加入 Tokenizer 中。
* 如果 `padding_side` 不是 `right`，請將其設定為 `right`。

In [13]:
# Add pad_token to the tokenizer
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
  print('=== 設定 Padding Token ===')
  pprint(tokenizer)
# Make sure padding_side is 'right'
if tokenizer.padding_side != 'right':
  tokenizer.padding_side = 'right'
  print('=== 設定 Padding Side ===')
  pprint(tokenizer)

### 載入預訓練模型

由於 GPU 記憶體有限，我們將使用半精度進行模型 Fine-tuning。這邊需要留意，使用半精度進行 Fine-tuning 時，`TrainingArguments` 中的 `adam_epsilon` 需要設定為 `1e-4`。預設的 `adam_epsilon` 是 `1e-8`，這個值在半精度訓練時會出現問題。

In [14]:
model = T5ForConditionalGeneration.from_pretrained(
  config.model_name,
  torch_dtype=config.torch_dtype,
  # 這個參數用於優化內存使用，減少模型加載時的 CPU 內存佔用，特別是在內存有限的環境中非常有用。
  low_cpu_mem_usage=True,
).to(device)

In [ ]:
pprint(model)

這是一個典型的 Encoder-Decoder 模型。

```json
T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    ...
  )
  (decoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    ...
  )
  (lm_head): Linear(in_features=512, out_features=32128, bias=False)
)    
```

### 詠唱格式化 (Prompt Formatting)

定義詠唱 (Prompt) 格式，我們將創建一個格式化函數。

請注意，這次我們指定 `add_generation_prompt` 為 `True`，表示回應開始的標記。這確保了當模型生成文本時，它會寫出機器人的回應，而不是做一些意想不到的事情，比如繼續用戶的訊息。請記住，聊天模型仍然只是語言模型，它們被訓練來玩文字接龍，而聊天對它們來說只是一種特殊的文本！你需要用適當的控制標記來引導它們，讓它們知道應該做什麼。

In [16]:
system_message = 'Mask the personal identifiable information:'

def instruction_formatter(x, tokenize: bool = False):
  input = f"{system_message} {x['unmasked_text']}"
  if tokenize:
    return tokenizer(
      [input],
      max_length=tokenizer.model_max_length,
      truncation=True,
      padding=True,
      return_tensors='pt',
    ).to(device)
  else:
    return input


In [ ]:
# tokenize=False 代表不進行 Tokenize，直接回傳原始文字
input = instruction_formatter(dataset['test'][0], tokenize=False)
pprint(input)

In [ ]:
# tokenize=True 代表進行 Tokenize，回傳 Tokenize 後的 ID 及 attention mask tensors
tokenized_input = instruction_formatter(dataset['test'][0], tokenize=True)
pprint(tokenized_input)

Tokenizer 回傳內容包含兩個主要部分：`input_ids` 和 `attention_mask`。以下是詳細解釋：

* input_ids: 是一個張量 (tensor)，包含了輸入文本的 token IDs。這些 IDs 是由 tokenizer 將文本轉換為數字表示後得到的。

* attention_mask: 同樣是一個張量，用於指示模型應該關注哪些位置。值為 1 的位置表示應該關注，值為 0 的位置表示應該忽略。在這個例子中，`attention_mask` 的值全為 1，表示模型應該關注所有位置。

In [ ]:
# 透過 Tokenizer 的 decode 方法將 ID 轉換回文字，並列顯示出來
for id in tokenized_input['input_ids'][0]:
  print(f'{id} -> {tokenizer.decode([id])}')

### Fine-tuning 前的表現

#### 單筆演示生成回應

In [ ]:
# 透過預訓練模型生成回應
output_ids = model.generate(
  **tokenized_input,
  temperature=config.temperature,
  max_new_tokens=config.max_new_tokens,
  repetition_penalty=config.repetition_penalty,
)

In [ ]:
output_ids

In [22]:
# 將 output_ids 轉換為文字
output = tokenizer.decode(
  output_ids[0],
  skip_special_tokens=False, # 決定是否跳過特殊 token（例如，開始和結束標記）。
)

In [ ]:
pprint(output)

#### 批次處理模型表現

初步了解如何生成模型的回應，我們將定義一個 `generate()` 函數來生成模型的回應。這個函數接受一個輸入文本，並生成模型的回應。藉由這個函數，我們可以批次處理資料。


In [24]:
# 將以上程式碼整理成一個函式，方便我們批次處理資料
def generator(x, model):
  tokenized_input = instruction_formatter(x, tokenize=True)
  output_ids = model.generate(
    **tokenized_input,
    temperature=config.temperature,
    max_new_tokens=config.max_new_tokens,
    repetition_penalty=config.repetition_penalty,
  )
  return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
# 這個步驟可能會花費一些時間，所以我們只處理前 first_n_data 筆資料
first_n_dataset = dataset['test'].select(range(first_n_data))

# 透過預訓練模型生成回應，將其新增到 first_n_dataset 的 pt_response 欄位中
first_n_dataset = first_n_dataset.map(
  lambda x: {
    **x,
    "pt_response": generator(x, model),
  },
  batched=False,
)

In [ ]:
# 顯示預訓練模型預測結果
pd.set_option('display.max_colwidth', None)
pd.DataFrame(first_n_dataset)

## 訓練模型

### LoRA 的訓練策略 - 降維打擊

LoRA（Low-Rank Adaptation）是一種用於訓練大型語言模型的技術，旨在提高訓練效率並減少計算資源的需求。以下是為何需要透過LoRA訓練的一些原因：

* 降低計算成本：LoRA 通過將模型的權重矩陣分解為低秩矩陣，顯著減少了參數的數量，從而降低了計算成本和內存需求。

* 加速訓練速度：由於參數數量減少，LoRA 可以加速模型的訓練過程，使得在相同的硬件資源下能夠更快地完成訓練。

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png)

In [ ]:
# 查看預訓練模型可訓練的參數量，其數量相當龐大，所以需要透過 Low Rank Adaptation (LoRA) 來降低參數量
print('Parameters: {:,}, Trainable Parameters: {:,}'.format(
  model.num_parameters(),
  model.num_parameters(only_trainable=True)))

#### LoRA 配置

* `task_type`: TaskType.CAUSAL_LM 指定任務類型為因果語言模型 (Causal Language Model)。

* `rank`: 是低秩矩陣的秩(rank)，它決定了 LoRA 層的參數數量。較低的 `r` 值意味著較少的參數，從而減少了模型的計算和存儲需求。具體來說，LoRA 通過將全連接層的權重矩陣分解為兩個低秩矩陣來實現參數高效化。`r` 值越小，這兩個低秩矩陣的維度越小，這個練習我們採用 128。

* `lora_alpha`: 是一個縮放因子，用於調整 LoRA 層的輸出。它控制了低秩矩陣的影響力。較高的 `lora_alpha` 值會增加 LoRA 層的影響力，也就是說值越高，越容易把大模型既有的能力給覆蓋掉。具體來說，LoRA 層的輸出會乘以這個縮放因子，這個練習我們採用常見的比例為 `rank` 的兩倍。

* `lora_dropout`: 是一個丟棄率，用於在訓練過程中隨機丟棄 LoRA 層的一部分輸出。這有助於防止過擬合，並提高模型的泛化能力。例如，`lora_dropout` 設置為 0.1 表示在每次前向傳播中，有 10% 的 LoRA 層輸出會被隨機設置為零。

* `target_module`: 指定了應用 LoRA 的目標模塊。這通常是模型中的某些特定層或子模塊，例如 Transformer 模型中的注意力層，可以透過 `model.named_parameters` 查看。通過指定 `target_module`，你可以靈活地選擇在哪些層應用 LoRA，以便在保持模型性能的同時減少參數數量。

> 廣為周知的模型當未指定 `target_module`，透過 `get_peft_model` 加載 Lora 適配模型時，會自動設定。
> 可以先嘗試不指定，若出現錯誤再試著設定注意力相關的參數層。


In [ ]:
# LoRA 配置
lora_config = LoraConfig(
  task_type=TaskType.SEQ_2_SEQ_LM,
  r=config.rank,
  lora_alpha=config.lora_alpha,
  lora_dropout=config.lora_dropout,
  # target_modules=['v', 'q'], # 有別於 Phi3ForCausalLM 需要指定 target_modules, 這次 LoRA 可以自動判斷
)

pprint(lora_config)

#### 加載 LoRA 適配模型

搭配預訓模型及 LoRA 配置，我們可以加載 LoRA 適配模型。我們可以觀察受到降維影響的模型層。

In [ ]:
# 加載 LoRA 適配模型
peft_model = get_peft_model(
  model, # 預訓練模型
  lora_config, # LoRA 配置
)

In [ ]:
pprint(lora_config)

#### LoRA 適配模型

加載 LoRA 適配模型後, 觀察受 LoRA 影響的模型參數

In [ ]:
peft_model

雖然我們沒有特別指定 target_module，但是在這個例子中，我們使用的是 T5 模型，因此 Ｑ 及 Ｖ 注意力層受到 LoRA 的影響。

```json
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=128, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=128, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): ParameterDict()
                    (lora_magnitude_vector): ModuleDict()
                  )
                  (k): Linear(in_features=512, out_features=384, bias=False)
                  (v): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=128, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=128, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): ParameterDict()
                    (lora_magnitude_vector): ModuleDict()
                  )
                  (o): Linear(in_features=384, out_features=512, bias=False)
                  (relative_attention_bias): Embedding(32, 6)
                )
```              

#### 調整 LoRA 精度

LoRA 適配模型的精度是 `torch.float32`，我們可以透過 `model.half()` 將其轉換為半精度。

In [ ]:
# 獲取 LoRA 模型參數名稱及型態，確認是否使用半精度浮點數
for name, param in peft_model.named_parameters():
  print(f'{name}: {param.dtype}')

可以發現除了預訓練模型的權重是半精度外，LoRA 適配模型的權重仍然是全精度。

```shell
base_model.model.encoder.block.0.layer.0.SelfAttention.q.base_layer.weight: torch.float16
base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight: torch.float32
base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_B.default.weight: torch.float32
base_model.model.encoder.block.0.layer.0.SelfAttention.k.weight: torch.float16
base_model.model.encoder.block.0.layer.0.SelfAttention.v.base_layer.weight: torch.float16
base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_A.default.weight: torch.float32
base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight: torch.float32
base_model.model.encoder.block.0.layer.0.SelfAttention.o.weight: torch.float16
```

In [33]:
if config.torch_dtype == torch.float16 or config.torch_dtype == torch.bfloat16:
  peft_model = peft_model.half() # 轉換為半精度浮點數

In [ ]:
# 獲取 LoRA 模型參數名稱及型態，確認是否使用半精度浮點數
for name, param in peft_model.named_parameters():
  print(f'{name}: {param.dtype}')

經過 `model.half()` 轉換後，LoRA 適配模型的權重也變成半精度。

```shell
base_model.model.encoder.block.0.layer.0.SelfAttention.q.base_layer.weight: torch.float16
base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight: torch.float16
base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_B.default.weight: torch.float16
base_model.model.encoder.block.0.layer.0.SelfAttention.k.weight: torch.float16
base_model.model.encoder.block.0.layer.0.SelfAttention.v.base_layer.weight: torch.float16
base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_A.default.weight: torch.float16
base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight: torch.float16
base_model.model.encoder.block.0.layer.0.SelfAttention.o.weight: torch.float16
```

訓練參數量也從原先 247M 大大減少為 14M。

In [ ]:
# 查看可訓練的參數量
peft_model.print_trainable_parameters()

### 資料預處理

在訓練模型之前，我們需要對資料進行預處理。這包括將文本轉換為模型可以理解的格式，包含輸入及輸出標籤 (Label)。

#### 定義預處理函數

In [36]:
def preprocess_function(dataset):
  inputs = [ f'{system_message} {q}' for q in dataset['unmasked_text'] ]
  tokenized_inputs = tokenizer(
    inputs,
    max_length=tokenizer.model_max_length,
    truncation=True,
    padding=True,
    return_tensors='pt',
  )
  # Temporarily sets the tokenizer for encoding the targets.
  # Useful for tokenizer associated to sequence-to-sequence models that need a slightly different processing for the labels.
  with tokenizer.as_target_tokenizer():
    input_labels = tokenizer(
      dataset['masked_text'],
      max_length=tokenizer.model_max_length,
      truncation=True,
      padding=True,
      return_tensors='pt',
    )
  # Replace all PAD tokens with -100 after eos_token_id for the labels
  input_labels['input_ids'] = input_labels['input_ids'].masked_fill(
    input_labels['input_ids'] == tokenizer.pad_token_id,
    -100,
  )
  tokenized_inputs['labels'] = input_labels['input_ids']
  return tokenized_inputs

In [ ]:
pprint(preprocess_function(dataset['test'][:first_n_data]))

#### 批次處理資料

In [ ]:
tokenized_dataset = dataset.map(
  preprocess_function,
  batched=True,
  remove_columns=dataset['train'].column_names, # 移除不必要的資料
)

### 資料校對器 (Data Collator)

在微調語言模型時，使用 data collator 是為了有效地準備和處理批次數據。以下是使用 data collator 的幾個主要原因：

* 動態填充 (Dynamic Padding): 不同長度的序列需要填充到相同的長度，以便能夠在同一批次中進行處理。Data collator 可以自動計算每個批次的最大長度，並對序列進行適當的填充。

* 批次處理 (Batch Processing): Data collator 可以將多個樣本組合成一個批次，這樣可以更高效地利用計算資源，特別是在使用 GPU 或 TPU 時。

* 生成注意力掩碼 (Attention Masks): 在填充序列時，data collator 會生成相應的注意力掩碼 (attention masks)，以確保模型只關注實際的數據部分，而忽略填充部分。

* 簡化代碼 (Code Simplification): 使用 data collator 可以簡化數據處理的代碼，減少手動處理數據的繁瑣步驟，讓開發者專注於模型設計和訓練。

總之，data collator 在微調語言模型時提供了便利和效率，確保數據能夠以一致且高效的方式進行處理。

在這邊我們使用 `DataCollatorForSeq2Seq` 是一個專門用於 BART 或 T5 這類 Seq2Seq 模型的數據整理器。

In [ ]:
# 設定 DataCollatorForCompletionOnlyLM
data_collator = DataCollatorForSeq2Seq(
  tokenizer=tokenizer,
  model=peft_model,
)

In [ ]:
# 展示 DataCollatorForSeq2Seq 的輸出
features = [tokenized_dataset["test"][:first_n_data]]
batch = data_collator(features)
pprint(batch)

這裡要注意的主要是第一個例子比第二個例子要長，所以第二個例子的 input_ids 和 attention_mask 已經在右側填充了一個 [PAD] 標記（其 ID 是 0）。 類似地，我們可以看到 labels 已用 -100 填充，以確保填充標記被損失函數忽略。最後，我們可以看到一個新的 `decoder_input_ids`，它通過在第一個條目中插入 [PAD] 標記將標籤向右移動。

我們終於擁有了訓練所需的所有的前期準備！我們現在只需要使用標準參數實例化訓練器。

### 模型評估函數

在訓練過程中包含度量標準通常有助於評估模型的性能。您可以使用 Evaluate 庫快速加載評估方法。

BLEU 和 ROUGE 分數都是在機器翻譯任務中廣泛使用的重要評估指標，但它們側重的方面不同，BLEU 側重於精確度，而 ROUGE 側重於召回率。

* `BLEU（Bilingual Evaluation Understudy）`: BLEU 分數側重於精確度。這些任務的主要目標是自動將文本從一種語言翻譯成另一種語言。BLEU 分數量化了機器翻譯的文本與參考翻譯之間的相似性。這種測量是使用 n-gram 進行的。

* `ROUGE（Recall-Oriented Understudy for Gisting Evaluation）`: ROUGE 分數側重於召回率。它將自動生成的摘要或翻譯與一個或多個參考進行比較。ROUGE 分數範圍從 0 到 1，反映了機器生成的摘要與參考之間的相似性，分數越高表示相似性越大。

> `use_stemmer` 這個參數指定是否在計算度量標準時使用詞幹提取（stemming）。
> 詞幹提取是將單詞還原為其詞幹形式（例如，將 "running" 還原為 "run"），這有助於提高評估的準確性，特別是在處理不同形式的單詞時。

In [ ]:
metric = evaluate.load(config.eval_metric)

def compute_metrics(eval_pred):
    # Unpack predictions and labels from the input
    predictions, labels = eval_pred

    # Apply the tokenizer to decode the predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels to the tokenizer pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Apply the tokenizer to decode the labels
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute evaluation metrics for rouge
    if config.eval_metric == 'rouge':
        result = metric.compute(
            predictions=decoded_preds,
            references=decoded_labels,
            use_stemmer=True,
        )
        return {k: round(v, 4) for k, v in result.items()}
    elif config.eval_metric == 'bleu':
        result = metric.compute(
            predictions=decoded_preds,
            references=decoded_labels,
        )
        return {"bleu": result["bleu"]}


### 訓練參數設定

用於設定訓練過程中的各種參數，如學習率、批次大小、梯度累積步數、訓練 epoch 數、權重衰減等。

* `output_dir` 指定了訓練輸出的目錄。
* `eval_strategy` 和 `save_strategy` 設定為 'epoch'，表示每個 epoch 都會進行評估和儲存。
* `load_best_model_at_end` 設定為 `True`，表示訓練結束後會載入最佳模型。
* `report_to` 設定為 'none'，禁用了 wandb 報告。
* `predict_with_generate` 設定為 `True`，表示在評估過程中使用生成的文本進行預測。
* `adam_epsilon` 設定了 Adam 優化器的 epsilon 值。
* `save_total_limit` 設定了最多儲存 5 個 checkpoints。

In [42]:
training_args = Seq2SeqTrainingArguments(
  output_dir='sample_data/train_output_pii_masking', # 訓練輸出目錄
  learning_rate=config.lr, # 學習率
  per_device_train_batch_size=config.batch_size, # 每個設備的訓練批次大小
  per_device_eval_batch_size=config.batch_size, # 每個設備的評估批次大小
  gradient_accumulation_steps=config.gradient_accumulation_steps, # 梯度累積步數
  num_train_epochs=config.epochs, # 訓練的總 epoch 數
  weight_decay=config.weight_decay, # 權重衰減
  eval_strategy='epoch', # 每個 epoch 評估一次
  save_strategy='epoch', # 每個 epoch 儲存一次
  load_best_model_at_end=True, # 訓練完後載入最佳模型
  report_to='none', # 禁用 wandb 報告 (Colab 環境預設需要 wandb)
  predict_with_generate=True, # 對評估數據集生成文字，針對生成文字計算指標
  adam_epsilon=config.adam_epsilon, # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  save_total_limit=5, # 最多儲存 5 個 checkpoints
)

### 訓練器初始化

用於初始化訓練器，並開始訓練模型。

* `model` 是要訓練的模型。
* `tokenizer` 是用於處理文本的分詞器。
* `train_dataset` 和 `eval_dataset` 是訓練和評估數據集。
* `data_collator` 是用於整理數據的數據整理器。
* `compute_metrics` 是用於計算度量標準的函數。

In [ ]:
trainer = Seq2SeqTrainer(
    model=peft_model, # 要訓練的模型
    tokenizer=tokenizer, # 使用的分詞器
    args=training_args, # 訓練參數
    train_dataset=tokenized_dataset['train'], # 訓練數據集
    eval_dataset=tokenized_dataset['validation'], # 評估數據集
    data_collator=data_collator, # 數據整理器
    compute_metrics=compute_metrics, # 計算指標
)

### 開始訓練

In [ ]:
# 開始訓練，這可能需要一些時間
trainer.train()

訓練完成後，您可以通過運行 `Trainer.evaluate()` 查看最終的分數:

In [ ]:
trainer.evaluate()

#### 保存 LoRA 模型參數

In [46]:
# 保存 Lora 参数
peft_model.save_pretrained(
  config.saved_lora_path,
)

#### 保存 Tokenizer

In [ ]:
# 保存 Tokenizer
tokenizer.save_pretrained(config.saved_model_path)

### 釋放資源

In [ ]:
# import garbage collector
import gc

# 釋放 GPU 記憶體
del trainer
del tokenizer

peft_model.to('cpu')
del peft_model

torch.cuda.empty_cache()

gc.collect()

## 評估微調模型

### 載入微調分詞器 (Tokenizer)

從已經完成訓練的模型取得 Tokenizer，可以留意這個訓練時保存下來的 Tokenizer 仍保有訓練時的設定，包涵 `pad_token` 和 `padding_side`。

In [49]:
tokenizer = AutoTokenizer.from_pretrained(
  config.saved_model_path
)

In [ ]:
# 檢視 Tokenizer 是否存在 padding token 及 padding side 等資訊
pprint(tokenizer.pad_token)

In [ ]:
pprint(tokenizer.padding_side)

### 載入微調後模型

In [52]:
ft_model = PeftModel.from_pretrained(
  model, # 預訓練模型
  config.saved_lora_path, # LoRA 適配模型
  # 這個參數用於優化內存使用，減少模型加載時的 CPU 內存佔用，特別是在內存有限的環境中非常有用。
  low_cpu_mem_usage=True,
  torch_dtype=config.torch_dtype,
).to(device)

### Fine-tuning 後的表現

In [ ]:
# 透過微調模型生成回應，將其新增到 first_n_dataset 的 ft_response 欄位中
first_n_dataset = first_n_dataset.map(
  lambda x: {
    **x,
    "ft_response": generator(x, ft_model),
  },
  batched=False,
)

In [ ]:
# 顯示微調模型預測結果
pd.set_option('display.max_colwidth', None)
pd.DataFrame(first_n_dataset)

# (Optional) Download files from Colab workspace

In [ ]:
![[ ! -z "${COLAB_GPU}" ]] && tar cvzf saved_encoder_model.tgz sample_data/saved_encoder_model/
![[ ! -z "${COLAB_GPU}" ]] && tar cvzf saved_lora_model.tgz sample_data/saved_lora_model/

In [56]:
import os
if 'COLAB_GPU' in os.environ:
  from google.colab import files
  files.download('saved_encoder_model.tgz')
  files.download('saved_lora_model.tgz')